# Depression Prediction - MLOps

### Import Library

In [1]:
import os
import pandas as pd
import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.components import Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import warnings
warnings.filterwarnings("ignore")

### Set Variabel

In [2]:
PIPELINE_ROOT = "syahvan-pipeline"
SCHEMA_PIPELINE_NAME = "depression-tfdv-schema"
SERVING_MODEL_DIR = "serving_model_dir"

### Load Dataset

In [3]:
df = pd.read_csv("data/depression.csv")

df

,text,depression
0,we understand that most people who reply immed...,1
1,welcome to r depression s check in post a plac...,1
2,anyone else instead of sleeping more when depr...,1
3,i ve kind of stuffed around a lot in my life d...,1
4,sleep is my greatest and most comforting escap...,1
...,...,...
7726,is that snow,0
7727,moulin rouge mad me cry once again,0
7728,trying to shout but can t find people on the list,0
7729,ughh can t find my red sox hat got ta wear thi...,0


In [4]:
DATA_ROOT = "data"

In [5]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

### Load Dataset Menggunakan Example Gen

Load data yang telah digabungkan sebelumnya menggunakan Example Gen pada machine learning pipeline

In [6]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

Pada ExampleGen diatas dataset dibagi untuk proses training dan evaluasi dengan rasio 80:20

## Data Validation

### Membuat Summary Statistic pada Dataset

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Membuat Data Schema

In [9]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'depression',INT,required,,-
'text',BYTES,required,,-


Terlihat dari SchemaGen yang telah dibuat, terdapat dua feature yaitu `depression` dan `text`, `depression` disini untuk menentukan prediksi apakah `text` berupa kalimat yang menandakan depresi atau tidak. `depression` bertipe int dan `text` bertipe bytes dan masing-masing feature tersebut required untuk melakukan prediksi

### Mengidentifikasi Anomali pada Dataset

Melakukan validator dengan mendeteksi anomali pada data berdasarkan summary statistic dan schema pada data

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"],
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(example_validator.outputs["anomalies"])

Dapat dilihat data yang akan digunakan tidak ditemukan anomali, sehingga siap untuk digunakan untuk proses pipeline selanjutnya

## Data Preprocessing
Processing yang dilakukan hanya untuk merubah nama feature dan label yang telah di transform menjadi text_xf, depression_xf, dan transform feature kedalam format lowercase string dan untuk casting label kedalam format int64 untuk memastikan tipe data pada label

In [13]:
TRANSFORM_MODULE_FILE = "depression_transform.py"

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}
import string
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "depression"
FEATURE_KEY = "text"


def transformed_name(key):
    return f"{key}_xf"


def preprocessing_fn(inputs):
    outputs = dict()
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(
        inputs[FEATURE_KEY]
    )
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Writing depression_transform.py


In [15]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: syahvan-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\a1bd5abb82424442b53995a6c12620c7\assets


INFO:tensorflow:Assets written to: syahvan-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\a1bd5abb82424442b53995a6c12620c7\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Tuner Hyperparameter
Melakukan tuning hyperparameter pada arsitektur model yang digunakan untuk mendapatkan hyperparameter terbaik.

In [16]:
TUNER_MODULE_FILE = "depression_tuner.py"

In [17]:
%%writefile {TUNER_MODULE_FILE}
import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from typing import NamedTuple, Dict, Text, Any
from keras_tuner.engine import base_tuner
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs


LABEL_KEY = "depression"
FEATURE_KEY = "text"
NUM_EPOCHS = 5

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=10,
)


def transformed_name(key):
    return f"{key}_xf"


def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=32):
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    
    return dataset


def model_builder(hp, vectorizer_layer):
    num_hidden_layers = hp.Choice(
        "num_hidden_layers", values=[1, 2]
    )
    embed_dims = hp.Int(
        "embed_dims", min_value=16, max_value=128, step=32
    )
    lstm_units= hp.Int(
        "lstm_units", min_value=32, max_value=128, step=32
    )
    dense_units = hp.Int(
        "dense_units", min_value=32, max_value=256, step=32
    )
    dropout_rate = hp.Float(
        "dropout_rate", min_value=0.1, max_value=0.5, step=0.1
    )
    learning_rate = hp.Choice(
        "learning_rate", values=[1e-2, 1e-3, 1e-4]
    )

    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string
    )
    
    x = vectorizer_layer(inputs)
    x = layers.Embedding(input_dim=5000, output_dim=embed_dims)(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)

    for _ in range(num_hidden_layers):
        x = layers.Dense(dense_units, activation=tf.nn.relu)(x)
        x = layers.Dropout(dropout_rate)(x)
    
    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["binary_accuracy"],
    )
    
    return model


def tuner_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, NUM_EPOCHS)
    eval_set = input_fn(fn_args.eval_files[0], tf_transform_output, NUM_EPOCHS)

    vectorizer_dataset = train_set.map(
        lambda f, l: f[transformed_name(FEATURE_KEY)]
    )

    vectorizer_layer = layers.TextVectorization(
        max_tokens=5000,
        output_mode="int",
        output_sequence_length=500,
    )
    vectorizer_layer.adapt(vectorizer_dataset)
    
    tuner = kt.Hyperband(
        hypermodel=lambda hp: model_builder(hp, vectorizer_layer),
        objective=kt.Objective('binary_accuracy', direction='max'),
        max_epochs=NUM_EPOCHS,
        factor=3,
        directory=fn_args.working_dir,
        project_name="kt_hyperband",
    )
    
    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [early_stopping_callback],
            "x": train_set,
            "validation_data": eval_set,
            "steps_per_epoch": fn_args.train_steps,
            "validation_steps": fn_args.eval_steps,
        },
    )

Writing depression_tuner.py


In [18]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=180),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=40),
)
interactive_context.run(tuner)

Trial 10 Complete [00h 06m 05s]
binary_accuracy: 0.9501736164093018

Best binary_accuracy So Far: 0.995312511920929
Total elapsed time: 00h 45m 06s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in syahvan-pipeline\.temp\6\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
num_hidden_layers: 2
embed_dims: 112
lstm_units: 32
dense_units: 224
dropout_rate: 0.30000000000000004
learning_rate: 0.001
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.995312511920929
Trial summary
Hyperparameters:
num_hidden_layers: 1
embed_dims: 48
lstm_units: 64
dense_units: 96
dropout_rate: 0.2
learning_rate: 0.01
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9930555820465088
Trial summary
Hyperparameters:
num_hidden_layers: 1
embed_dims: 112
lstm_units: 96
dense_units: 128
dropout_rate: 0.5
learning_rate: 0.01
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0001
Score: 0.9821180701255798
Trial summary
Hyperparameters:
num_hidden_layers: 2
embed_dims: 16
lstm_units: 96
dense_units: 128
dropout_rate: 0.30000000000000004
learning_rate: 0.01
tuner/epochs: 5
tun

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

## Training Model
Melatih model dengan menggunakan hyperparameter yang telah dituning sebelumnya

In [19]:
TRAINER_MODULE_FILE = "depression_trainer.py"

In [20]:
%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_hub as hub
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs


LABEL_KEY = "depression"
FEATURE_KEY = "text"


def transformed_name(key): 
    return f"{key}_xf"


def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=32):
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    
    return dataset


def model_builder(vectorizer_layer, hp):
    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string
    )

    x = vectorizer_layer(inputs)
    x = layers.Embedding(input_dim=5000, output_dim=hp["embed_dims"])(x)
    x = layers.Bidirectional(layers.LSTM(hp["lstm_units"]))(x)

    for _ in range(hp["num_hidden_layers"]):
        x = layers.Dense(hp["dense_units"], activation=tf.nn.relu)(x)
        x = layers.Dropout(hp["dropout_rate"])(x)

    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp["learning_rate"]),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )
    
    model.summary()
    
    return model


def _get_serve_tf_example_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec
        )
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using transformed features
        return model(transformed_features)
    
    return serve_tf_examples_fn


def run_fn(fn_args: FnArgs):
    hp = fn_args.hyperparameters["values"]
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        patience=10,
    )
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        save_best_only=True,
    )
    callbacks = [
        tensorboard_callback, 
        early_stopping_callback, 
        model_checkpoint_callback
    ]
    
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    train_set = input_fn(
        fn_args.train_files, tf_transform_output, hp["tuner/epochs"]
    )
    eval_set = input_fn(
        fn_args.eval_files, tf_transform_output, hp["tuner/epochs"]
    )
    
    vectorizer_dataset = train_set.map(
        lambda f, l: f[transformed_name(FEATURE_KEY)]
    )
    
    vectorizer_layer = layers.TextVectorization(
        max_tokens=5000,
        output_mode="int",
        output_sequence_length=500,
    )
    vectorizer_layer.adapt(vectorizer_dataset)

    model = model_builder(vectorizer_layer, hp)
    
    model.fit(
        x=train_set,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_set,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks,
        epochs=hp["tuner/epochs"],
        verbose=1,
    )
    
    signatures = {
        "serving_default": _get_serve_tf_example_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name="examples",
            )
        )
    }
    
    model.save(
        fn_args.serving_model_dir, 
        save_format="tf", 
        signatures=signatures
    )

Writing depression_trainer.py


In [21]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=180),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=40),
)
interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 500)              0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 500, 112)          560000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               37120     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 224)               14560     
                                                                 
 dropout_1 (Dropout)         (None, 224)               0   

INFO:tensorflow:Assets written to: syahvan-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: syahvan-pipeline\Trainer\model\7\Format-Serving\assets


180/180 [==============================] - 63s 337ms/step - loss: 0.2767 - binary_accuracy: 0.8814 - val_loss: 0.0828 - val_binary_accuracy: 0.9719
Epoch 2/5
180/180 [==============================] - ETA: 0s - loss: 0.0632 - binary_accuracy: 0.9783
Epoch 2: val_binary_accuracy did not improve from 0.97188
180/180 [==============================] - 67s 372ms/step - loss: 0.0632 - binary_accuracy: 0.9783 - val_loss: 0.0919 - val_binary_accuracy: 0.9617
Epoch 3/5
180/180 [==============================] - ETA: 0s - loss: 0.0392 - binary_accuracy: 0.9872
Epoch 3: val_binary_accuracy improved from 0.97188 to 0.97344, saving model to syahvan-pipeline\Trainer\model\7\Format-Serving


INFO:tensorflow:Assets written to: syahvan-pipeline\Trainer\model\7\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: syahvan-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: syahvan-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Analisis dan Evaluasi Model
Menganalisa dan mengevaluasi model dengan menggunakan beberapa metric beserta nilai threshold pada metric tersebut

In [22]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id("Latest_blessed_model_resolver")

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [23]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key="depression")],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name="ExampleCount"),
            tfma.MetricConfig(class_name="AUC"),
            tfma.MetricConfig(class_name="TruePositives"),
            tfma.MetricConfig(class_name="FalsePositives"),
            tfma.MetricConfig(class_name="TrueNegatives"),
            tfma.MetricConfig(class_name="FalseNegatives"),
            tfma.MetricConfig(class_name="BinaryAccuracy", 
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={"value": 0.6},
                    ),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={"value": 1e-4},
                    ),
                ),
            ),
        ])
    ]
)

In [24]:
evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
)
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [25]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

## Export Model
Mengexport model menggunakan komponen pusher pada tensorflow extended agar dapat digunakan untuk proses deployment model.

In [26]:
pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join(
                SERVING_MODEL_DIR, "depression-detection-model"
            ),
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))